## 과제 2: SageMaker Experiments 사용

이 실습에서는 Amazon SageMaker Experiments를 사용하여 실험을 설정합니다. 구체적으로는 XGBoost를 사용하여 기계 학습 모델을 훈련하고 하이퍼파라미터 튜닝을 수행해 여러 하이퍼파라미터 설정을 테스트합니다. 이 테스트 결과를 토대로 더욱 정확한 모델을 생성한 후 모델 성능을 평가합니다.

### 과제 2.1: Amazon CodeGuru 보안 스캔 확장 프로그램 사용

이 과제에서는 정기적으로 노트북 스캔을 수행하도록 Amazon CodeGuru를 구성합니다. 

자동 코드 스캔은 기본적으로 비활성화되어 있습니다. 스캔은 정기적으로 자동 실행되거나 모든 코드 셀에서 수동으로 실행할 수 있습니다. 

1. 도구 모음에서 **Settings**를 선택합니다.
1. **Advanced Settings Editor**를 선택합니다.

SageMaker Studio에서 *Settings* 탭이 표시됩니다.

1. 왼쪽 탐색 창에서 **Amazon CodeGuru**를 선택합니다.
1. **Auto scans** 드롭다운 메뉴에서 **Enabled**를 선택합니다.

자동 스캔은 활성화되면 기본적으로 240초마다 실행됩니다. 이 실습의 기본값을 수락합니다. 

1. *Settings* 탭을 닫고 *lab_6_ko_kr.ipynb* 노트북이 있는 탭으로 돌아갑니다.




### 과제 2.2: 환경 설정

모델 훈련을 시작하기 전에 필요한 종속 항목을 모두 설치하십시오.


SageMaker Experiments의 특성에 관하여 자세히 알아보려면 [Amazon SageMaker Experiments를 사용하여 기계 학습 관리](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html#experiments-features)를 참조하세요.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.experiments.run import Run, load_run
#from sagemaker.utils import unique_name_from_base  #could be used instead of the date-time append approach, to create a unique Experiment name.
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlasms'

CodeGuru 보안 스캔은 노트북의 모든 코드 셀 내에서 수동으로 시작할 수 있습니다. 

수동 코드 스캔을 시작합니다.

1. 이전 코드 셀을 선택합니다.
1. 마우스 오른쪽 버튼을 클릭해 컨텍스트 메뉴를 열고 **Run CodeGuru scan**을 선택합니다.

**CodeGuru: Scan in progress**는 스캔이 실행되는 동안 SageMaker Studio 환경의 하단 가장자리에 표시됩니다. 스캔이 완료되면 사라집니다.

1. 이전 코드 셀을 선택합니다.
1. 마우스 오른쪽 버튼을 클릭해 컨텍스트 메뉴를 열고 **Show diagnostics panel**을 선택합니다.

SageMaker Studio에서 환경 하단의 *Diagnostics Panel* 탭이 열립니다.

*Diagnostics Panel*에서 "문제: 노트북 모범 사례 위반. 제안된 문제 해결 방법: 노트북의 첫 번째 셀 이후에 import 문이 발견되었습니다"라는 메시지가 표시됩니다. 이 실습에서는 이 문제를 해결할 필요가 없습니다.

사용 중인 코드에 대해 CodeGuru 스캔이 제공할 수 있는 권장 사항의 유형에 관한 자세한 내용은 [Amazon CodeGuru 확장 프로그램 권장 사항 유형](https://docs.aws.amazon.com/codeguru/latest/security-ug/recommendation-types-extension.html)을 참조하세요.

1. *Diagnostics Panel* 탭을 닫습니다.

그런 다음, 데이터 집합을 가져옵니다.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

이 데이터 집합은 훈련(70%), 검증(20%), 검정(10%) 데이터 집합으로 분할합니다. 훈련 및 검증 데이터 집합은 훈련 중에 있습니다. 검정 데이터 집합은 배포 후의 모델 평가에 사용됩니다.

Amazon SageMaker를 사용하여 훈련하려면 데이터 집합을 libSVM 또는 CSV 형식으로 변환해야 합니다. 이 실습에서는 훈련에 CSV 형식을 사용합니다. 

XGBoost 알고리즘에 관한 자세한 내용은 [XGBoost 알고리즘](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)을 참조하세요. 
XGBoost 알고리즘용 입력/출력 인터페이스에 관한 자세한 내용은 [XGBoost 알고리즘용 입력/출력 인터페이스](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost)를 참조하세요.


In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

*train_data.csv* 및 *validation_data.csv*라는 두 개의 데이터 집합 파일을 생성했습니다. 
이제 Amazon Simple Storage Service(Amazon S3)에 이 두 데이터 집합 파일을 업로드합니다.

In [ ]:
#upload-dataset
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## 과제 2.3: 실험 생성 및 초기 훈련 작업 실행

SageMaker Experiments를 사용하여 다양한 훈련 구성 요소를 통해 ML 모델 훈련 실험을 구성, 추적, 비교, 평가합니다. SageMaker Experiments에 관한 자세한 내용은 [Amazon SageMaker Experiments로 기계 학습 관리](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html)를 참조하세요. SageMaker Experiments에서 이러한 구성 요소에는 데이터 집합, 알고리즘, 하이퍼파라미터, 지표 등이 포함됩니다. 

이 과제에서는 다음 작업을 완료합니다.
- Amazon SageMaker Studio에서 실험 생성 및 추적
- 실행을 생성하여 입력, 파라미터, 지표를 추적

먼저 해당 실험의 이름을 만들고 설명을 입력합니다.

In [ ]:
#create unique experiment name
create_date = strftime("%m%d%H%M")

lab_6_experiment_name = "lab-6-{}".format(create_date)
description = "Using SageMaker Experiments with the Adult dataset."

그런 다음, 실행 이름과 태그에 대한 선택적 값을 정의합니다.

In [ ]:
# create initial run_name
run_name = "lab-6-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-6', 'Value': 'lab-6-run'}]

print(f"Experiment name - {lab_6_experiment_name},  run name - {run_name}")

### 과제 2.4: XGBoost 알고리즘을 사용하여 모델 훈련 및 튜닝

이전 과제에서 훈련에 사용할 수 있도록 실험을 설정했습니다. 훈련이 완료되면 SageMaker Studio에서 결과를 분석할 수 있습니다. 이 과제에서는 다음을 수행합니다. 

- XGBoost 모델 훈련
- SageMaker Studio에서 실험 분석
- 하이퍼파라미터를 사용하여 모델 튜닝
- SageMaker Studio에서 튜닝 결과 분석

### 과제 2.5: XGBoost 모델 훈련

이제 방금 생성한 실험과 XGBoost 알고리즘을 사용하여 모델을 훈련합니다. 

설정한 하이퍼파라미터는 다음과 같습니다.
- **eta**: 과대적합 방지를 위해 업데이트에서 사용되는 단계 크기 축소 파라미터입니다. 각 부스팅 단계를 수행한 후 새 특성의 가중치를 직접 가져올 수 있습니다. 
- **gamma**: 트리의 리프 노드에서 추가 파티션을 만드는 데 필요한 최소 손실 감소 파라미터입니다. 이 파라미터의 값이 클수록 알고리즘은 예상 결과와 가까운 결과를 반환합니다.
실제로 eta 파라미터는 부스팅 프로세스에서 예상 결과와 가까운 결과를 반환하도록 특성 가중치를 축소합니다.
- **max_depth**: 트리의 최대 깊이입니다. 이 값을 높이면 모델이 더 복잡해지며 과대적합이 발생할 가능성이 높아집니다.
- **min_child_weight**: 하위 항목에 필요한 인스턴스 가중치의 최소 합계(hessian)입니다. 트리 파티션 단계를 수행했는데 생성된 리프 노드의 인스턴스 가중치 합계가 min_child_weight보다 작으면 빌드 프로세스에서는 추가 분할을 수행하지 않습니다. 선형 회귀 모델에서 이 값은 각 노드에 필요한 인스턴스의 최소 수에 해당합니다. 알고리즘은 크기가 클수록 예상 결과와 가까운 결과를 반환합니다.
- **num_round**: 부스팅에 사용되는 라운드(트리)의 수입니다. 트리 크기를 늘리면 모델 정확도는 높아질 수 있지만 과대적합 위험성도 높아집니다.
- **objective**: 학습 과제와 해당 학습 목표를 지정합니다.
- **subsample**: 훈련 인스턴스의 하위 샘플 비율입니다. 가령 이 파라미터를 0.5로 설정하면 XGBoost는 데이터 인스턴스의 절반을 무작위로 수집하여 트리를 확장합니다. 따라서 과대적합을 방지할 수 있습니다.
- **verbosity**: 인쇄 메시지의 세부 사항입니다. 유효한 값은 0(자동), 1(경고), 2(정보), 3(디버그)입니다.

훈련을 실행하려면 약 5-7분이 소요됩니다.

XGBoost 하이퍼파라미터에 관한 자세한 내용은 [XGBoost 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)를 참조하세요.

In [ ]:
from sagemaker import image_uris
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')

# initialize hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8
verbosity=0

hyperparameters = {
        "max_depth":max_depth,
        "eta":eta,
        "gamma":gamma,
        "min_child_weight":min_child_weight,
        "subsample":subsample,
        "verbosity":verbosity,
        "objective":objective,
        "num_round":num_round
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    EnableSageMakerMetricsTimeSeries=True,
    hyperparameters=hyperparameters,
    tags = run_tags
)


#Run the training job link to Experiment.
with Run(
    experiment_name=lab_6_experiment_name,
    run_name=run_name,
    tags=run_tags,
    sagemaker_session=sagemaker_session,
) as run:

    run.log_parameters({
                        "eta": eta, 
                        "gamma": gamma, 
                        "max_depth": max_depth,
                        "min_child_weight": min_child_weight,
                        "num_round": num_round,
                        "objective": objective,
                        "subsample": subsample,
                        "verbosity": verbosity
                       })
    
#    you may also specify metrics to log
#    run.log_metric(name="", value=x)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    ) 

### 과제 2.6: 튜닝 전에 모델 성능 평가

SageMaker Studio에서는 차트를 생성하여 훈련 작업을 평가할 수 있습니다. 예를 들어 lab-6 실험을 실행한 후 validation:logloss_max 값을 차트 형식으로 검토할 수 있습니다.

이 실습에서는 노트북에 추가 지표를 그릴 수 있습니다.

In [ ]:
#visualize-training-results-table
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
plt.show()

### 과제 2.7: 하이퍼파라미터를 사용하여 모델 튜닝

SageMaker Experiments를 사용하여 모델 훈련을 수행했습니다. 훈련 중에는 SageMaker가 하이퍼파라미터를 사용하도록 구성할 수도 있습니다. 그러면 훈련 대상 모델의 성능을 대폭 개선할 수 있습니다. SageMaker Studio에는 모델 훈련에 사용되는 다양한 공통 하이퍼파라미터 튜닝 옵션이 포함되어 있습니다. 훈련에 사용하는 데이터 집합에 따라 여러 파라미터를 테스트하는 효과는 달라질 수 있습니다. 그러나 최적의 모델을 만드는 데 상당한 시간과 노력이 필요할 수도 있습니다.

SageMaker 자동 모델 튜닝은 훈련을 최적화하기 위해 하이퍼파라미터 선택을 자동화합니다. 자동 모델 튜닝에 관한 자세한 내용은 [SageMaker를 사용하여 자동 모델 튜닝 수행](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)을 참조하세요. 이 기능을 사용하려면 튜닝하도록 선택하는 각 하이퍼파라미터의 범위(사용 가능한 값 목록)를 지정합니다. SageMaker 자동 모델 튜닝은 다양한 하이퍼파라미터 설정을 사용해 여러 훈련 작업을 자동으로 실행합니다. 그런 다음, 지정된 목표 지표에 따라 각 작업의 결과를 평가하고 이전 결과를 기준으로 하여 추가 시도를 위한 하이퍼파라미터 설정을 선택합니다. 각 튜닝 작업에서는 최대 훈련 작업 수를 지정합니다. 해당 작업 수에 도달하면 튜닝이 완료됩니다.

설정해야 할 하이퍼파라미터 범위는 다음과 같습니다.
- **alpha**: 가중치에 사용되는 L1 정규화 항입니다. 이 값을 늘리면 모델이 예상 결과와 가까운 결과를 반환합니다.
- **eta**: 과대적합 방지를 위해 업데이트에서 사용되는 단계 크기 축소 파라미터입니다. 각 부스팅 단계를 수행한 후 새 특성의 가중치를 직접 가져올 수 있습니다. 실제로 eta 파라미터는 부스팅 프로세스에서 예상 결과와 가까운 결과를 반환하도록 특성 가중치를 축소합니다.
- **max_depth**: 트리의 최대 깊이입니다. 이 값을 높이면 모델이 더 복잡해지며 과대적합이 발생할 가능성이 높아집니다.
- **min_child_weight**: 하위 항목에 필요한 인스턴스 가중치의 최소 합계(hessian)입니다. 트리 파티션 단계를 수행했는데 생성된 리프 노드의 인스턴스 가중치 합계가 min_child_weight보다 작으면 빌드 프로세스에서는 추가 분할을 수행하지 않습니다. 선형 회귀 모델에서 이 값은 각 노드에 필요한 인스턴스의 최소 수에 해당합니다. 알고리즘은 크기가 클수록 예상 결과와 가까운 결과를 반환합니다.
- **num_round**: 부스팅에 사용되는 라운드(트리)의 수입니다. 트리 크기를 늘리면 모델 정확도는 높아질 수 있지만 과대적합 위험성도 높아집니다.

튜닝을 완료하려면 약 5분이 소요됩니다.

XGBoost 하이퍼파라미터에 관한 자세한 내용은 [XGBoost 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)를 참조하세요.

In [ ]:
#tune-model
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# Setup the hyperparameter ranges
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 2),
    'eta': ContinuousParameter(0, 1),
    'max_depth': IntegerParameter(1, 10),
    'min_child_weight': ContinuousParameter(1, 10),
    'num_round': IntegerParameter(100, 1000)
}
# Define the target metric and the objective type (max/min)
objective_metric_name = 'validation:auc'
objective_type='Maximize'
# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type='Auto',
)

with load_run(sagemaker_session=sagemaker_session, experiment_name=lab_6_experiment_name, run_name=run_name) as run:
# Tune the model
    tuner.fit(
        inputs = data_inputs,
        job_name = lab_6_experiment_name,
    )
    run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
 
plt.show()
    

### 과제 2.8: 튜닝 후에 모델 성능 평가

SageMaker Studio에서는 차트를 생성하여 훈련 작업을 평가할 수도 있습니다. 예를 들어 lab-6-trial 훈련 작업을 실행한 후에는 목표 값 **validation:auc_max**를 차트 형식으로 확인할 수 있습니다.

![SageMaker Studio의 validation:error_max 차트 이미지](Task_2_3_4.png)

이 실습에서는 최적 튜닝 작업에서 결과를 확인하고 노트북에서 차트를 사용하여 해당 결과를 시각화합니다.

In [ ]:
#get_experiment_analytics 
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name+"-aws-tuning-job",
    sagemaker_session=Session(sess, sm),
)

run_component_analytics.dataframe()

In [ ]:
#visualize-tuning-results-auc-max
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    run_component_analytics.dataframe()["validation:auc - Max"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
else:
    run_component_analytics.dataframe()["validation:auc - Last"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
    
plt.show()

In [ ]:
#visualize-tuning-results-auc-max-scatter
N = 12
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Max"];
else:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Last"];
y = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["num_round"]

plt.scatter(x, y, alpha=0.5)
plt.title("auc_max by num_round")
plt.xlabel("validation:auc - Max")
plt.ylabel("num_round");
plt.show()

마지막으로 목표 지표를 기준으로 하여 최적 튜닝 작업을 인쇄합니다.

In [ ]:
#print-best
tuner.best_training_job()

### 과제 2.9: 기본 기능을 사용하여 SageMaker Studio로 실험 지표 그래프 작성

앞서 언급한 방법은 인라인 노트북 셀을 사용하여 실험 지표로 차트를 만듭니다. 추가 옵션은 SageMaker Studio 내 기능을 사용하여 일부 실험 지표를 구성하는 것입니다. 실험이 한 번 이상 실행되었으므로 이제 SageMaker Studio에서 새 막대 차트를 생성합니다.

다음 과제에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_6_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **lab_6_ko_kr.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 아티팩트를 살펴볼 때 지침을 표시할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 아티팩트 탐색을 완료한 후 **lab_6_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. **SageMaker Home** 아이콘을 선택합니다.
1. **Experiments**를 선택합니다.

SageMaker Studio에서 **Experiments** 탭이 표시됩니다.

1. *aws-tuning-job*으로 끝나는 실험을 선택합니다.

SageMaker Studio는 해당 실험에 포함된 **Runs** 목록을 표시합니다.

1. 열 헤더 행에서 **Name** 열의 체크 박스를 선택합니다.
1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Analyze</span>를 선택합니다.

SageMaker Studio에 **Run Analyze Chart** 탭이 표시됩니다.

1. 탭 하단의 차트 섹션에서 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">+ Add Chart</span>를 선택합니다.
1. **Bar**를 선택합니다.

SageMaker Studio에서 **Add Chart** 창이 표시됩니다.

1. **Y-axis**에서 **min_child_weight**를 선택합니다.
1. <span style="background-color:#73cdf9; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-radius:2px; border-width:3px; margin-right:5px; white-space:nowrap">Create</span>를 선택합니다.

실험에서 *run*당 *min_child_weight*를 보여주는 막대 차트가 이제 차트 섹션에 저장됩니다.

1. 이 프로세스를 반복하고 **train:auc** 지표에 대한 새 막대 차트를 생성합니다.
1. 이 프로세스를 반복하고 **validation:auc** 지표에 대한 새 막대 차트를 생성합니다.

### 마무리

축하합니다! Amazon SageMaker Experiments를 사용하여 모델 훈련과 튜닝을 수행했습니다. 다음 실습에서는 SageMaker Debugger를 사용하여 모델 훈련 중에 발생할 수 있는 문제에 관한 인사이트를 파악합니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **Conclusion** 부분을 계속 진행합니다.